In [1]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
import numpy as np

In [2]:
def prepare_data(path : str) -> tuple:
    files = glob.glob(f'{path}/data-*')
    scaler = MinMaxScaler()
    aggregated_data = pd.DataFrame()

    for file in files:
        current_data = pd.read_csv(file, delimiter='\t', header=None, names=['Date', 'Time', 'Type', 'Insulin'])
        aggregated_data = pd.concat([aggregated_data, current_data])

    aggregated_data = aggregated_data[['Date', 'Time', 'Insulin', 'Type']]

    columns_to_normalize = ['Insulin', 'Time', 'Date']
    aggregated_data['Insulin'] = aggregated_data['Insulin'].replace(["0Hi", "0Lo", "0''"], 0.0)
    aggregated_data['Time'] = pd.to_datetime(aggregated_data['Time'], format='%H:%M', errors='coerce').dt.time
    aggregated_data['Time'] = aggregated_data['Time'].apply(lambda x: x.hour * 60 + x.minute)
    aggregated_data['Date'] = pd.to_datetime(aggregated_data['Date'], format='%m-%d-%Y', errors='coerce')
    aggregated_data['Date'] = aggregated_data['Date'].dt.dayofyear
    aggregated_data = aggregated_data.dropna()

    aggregated_data[columns_to_normalize] = scaler.fit_transform(aggregated_data[columns_to_normalize])

    aggregated_data.to_csv('normalized_data.csv', index=False)

    features = aggregated_data[['Date', 'Time', 'Insulin']].to_numpy()
    labels = aggregated_data['Type'].to_numpy()

    features_train, features_test, labels_train, labels_test = tts(features, labels, test_size=0.2, random_state=42)

    return (features_train, features_test, labels_train, labels_test)


In [3]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)

        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)

        return self.final_output

    def backward(self, X, y, output):
        error = y - output
        output_delta = error * self.sigmoid_derivative(output)

        error_hidden = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = error_hidden * self.sigmoid_derivative(self.hidden_output)

        self.weights_hidden_output += self.hidden_output.T.dot(output_delta)
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True)

        self.weights_input_hidden += X.T.dot(hidden_delta)
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True)

    def train(self, X, y, epochs):
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)


In [ ]:
features_train, features_test, labels_train, labels_test = prepare_data('/home/oliwier/WSI_lab/lab_6/Diabetes-Data')

input_size = 3
hidden_size = 4
output_size = 1

# Initialize the neural network
nn = NeuralNetwork(input_size, hidden_size, output_size)

# Train the neural network
epochs = 10000
nn.train(features_train, labels_train, epochs)

# Test the neural network
predictions = nn.forward(features_test)